In [1]:
import pandas as pd
import numpy as np

import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from utils.dataset import FeatureCaptionDataset
from utils.models import CNNEncoder, RNNDecoder
from utils.helpers import get_tokenizer_vocab, collate_fn_pad

In [2]:
PATH = "dataset/"

In [3]:
df = pd.read_csv(PATH + "captions.txt")

In [4]:
df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [5]:
tokenizer, vocab = get_tokenizer_vocab(df)

In [6]:
dataset = FeatureCaptionDataset(PATH + "embeddings/", df, tokenizer, vocab)

In [7]:
dataiter = iter(dataset)
feats, cap = next(dataiter)

In [8]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, collate_fn=collate_fn_pad)

In [9]:
encoder_dim, decoder_dim, attention_dim, embed_dim, vocab_size = 512, 256, 128, 64, 500

In [10]:
# cnn_encoder = CNNEncoder(1536, encoder_dim)

In [11]:
# decoder = RNNDecoder(encoder_dim=encoder_dim, 
#                      decoder_dim=decoder_dim, 
#                      attention_dim=attention_dim, 
#                      embedding_dim=embed_dim, 
#                      vocab_size=vocab_size
#                     )

In [12]:
vocab_size = len(vocab)
print(vocab_size)

9213


In [13]:
feats, cap = next(iter(dataloader))

In [14]:
feats.shape, cap.shape

(torch.Size([2, 1536, 8, 8]), torch.Size([2, 18]))